In [22]:
import torch
import numpy as np

if torch.backends.mps.is_available():
    print("Training on GPU")
    device ="mps"
else:
    print("Training on CPU")
    device="cpu"

Training on GPU


In [17]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [18]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

num_workers = 0
batch_size = 20
valid_size = 0.2

transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5), (0.5))
            ])

train_data = datasets.CIFAR10('cifar_data/', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10('cifar_data', train=False, download=True, transform=transform)

num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size*num_train))
train_index, valid_index = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_index)
valid_sampler = SubsetRandomSampler(valid_index)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

Files already downloaded and verified
Files already downloaded and verified


In [32]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #Input to convolutional layer is 32x32x3 image
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        #convolutional layer(sees 16x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        #convolutional layer(sees 8x8x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        #max pooling layer
        self.pool = nn.MaxPool2d(2,2)
        #Linear layer (64*4*4 -> 500)
        self.fc1 = nn.Linear(64*4*4, 500)
        #Linear layer (500 -> 10)
        self.fc2 = nn.Linear(500, 10)
        #dropout layer
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        #sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        #flatten image input
        x = x.view(-1, 64*4*4)
        #add dropout layer
        x = self.dropout(x)
        #add first hidden layer with relu activation function
        x = F.relu(self.fc1(x))
        #add dropout layer
        x = self.dropout(x)
        #add second hidden layer with relu activation function
        x = self.fc2(x)
        return x        

In [33]:
model = Net()
model.to(device)

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [34]:
import torch.optim as optim

#Loss function
criterion = nn.CrossEntropyLoss()

#Optimization function
optimizer = optim.SGD(model.parameters(), lr=0.01)

#Training epochs
epochs = 30

#
valid_loss_min = np.Inf

for epoch in range(epochs):
    #Monitor training and validation loss
    train_loss = 0
    valid_loss = 0
    
    #TRAINING
    for data, target in train_loader:
        #Clear gradients
        optimizer.zero_grad()
        #Get the prediction
        output = model(data.to(device))
        #Calculate the loss
        loss = criterion(output, target.to(device))
        #back propogation
        loss.backward()
        #parameter update
        optimizer.step()
        #Update training loss
        train_loss += loss.item()*data.size(0)
    
    #VALIDATION
    for data, target in valid_loader:
        #Get the prediction
        output = model(data.to(device))
        #Calculate the loss
        loss = criterion(output, target.to(device))
        #Update validation loss
        valid_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    print("Epoch:{} \tTraining Loss:{} \tValidation Loss:{}".format(epoch+1, train_loss, valid_loss))
    
    if valid_loss < valid_loss_min:
        print("Validation loss decreased: ({:6f} --> {:6f}. Saving model...".format(valid_loss_min, valid_loss))
        torch.save(model.state_dict(), "model.pt")
        valid_loss_min = valid_loss

Epoch:1 	Training Loss:1.6961045527458192 	Validation Loss:0.3668908120155335
Validation loss decreased: (   inf --> 0.366891. Saving model...
Epoch:2 	Training Loss:1.3569602586269378 	Validation Loss:0.31991595726013183
Validation loss decreased: (0.366891 --> 0.319916. Saving model...
Epoch:3 	Training Loss:1.209359387111664 	Validation Loss:0.28671665382385253
Validation loss decreased: (0.319916 --> 0.286717. Saving model...
Epoch:4 	Training Loss:1.1113173503160476 	Validation Loss:0.2698587891817093
Validation loss decreased: (0.286717 --> 0.269859. Saving model...
Epoch:5 	Training Loss:1.0342878309726715 	Validation Loss:0.25393353950977327
Validation loss decreased: (0.269859 --> 0.253934. Saving model...
Epoch:6 	Training Loss:0.9693242420911788 	Validation Loss:0.23772096421718597
Validation loss decreased: (0.253934 --> 0.237721. Saving model...
Epoch:7 	Training Loss:0.9092762155771256 	Validation Loss:0.2300840200662613
Validation loss decreased: (0.237721 --> 0.230084. 

In [35]:
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [48]:
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

for data, target in test_loader:
    data = data.to(device)
    target = target.to(device)
    output = model(data)
    
    
    loss = criterion(output, target)
    
    test_loss += loss.item()*data.size(0)
    
    _, pred = torch.max(output, 1)
    
    correct = np.squeeze(pred.eq(target.to(device).data.view_as(pred)))
    
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].data.cpu()
        class_total[label] += 1
test_loss = test_loss/len(test_loader.dataset)
print("Test Loss:{}".format(test_loss))
    
for i in range(10):
    if class_total[i] > 0:
        print("Test Accuracy of %5s: %2d%% (%2d/%2d)" % (str(i), 
              100*class_correct[i]/class_total[i], class_correct[i], class_total[i]))
    else:
        print("Test Accuracy of %5s: N/A (no training examples)" % (classes[i]))
print('\nTest Accuracy (overall):%2d%% (%2d/%2d)' % (100. * np.sum(class_correct)/np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))

Test Loss:0.8962790278792381
Test Accuracy of     0: 79% (793/1000)
Test Accuracy of     1: 85% (857/1000)
Test Accuracy of     2: 57% (573/1000)
Test Accuracy of     3: 49% (496/1000)
Test Accuracy of     4: 62% (624/1000)
Test Accuracy of     5: 62% (620/1000)
Test Accuracy of     6: 79% (796/1000)
Test Accuracy of     7: 77% (774/1000)
Test Accuracy of     8: 83% (832/1000)
Test Accuracy of     9: 70% (700/1000)

Test Accuracy (overall):70% (7065/10000)
